In [34]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import mannwhitneyu, ttest_ind

# Function to divide into two classes

In [35]:
def std_classification(df):
    temp = zscore(df)
    temp = [0 if l<=0.0 else 1 for l in temp.values]
    return temp

# Load Test Scores

In [36]:
mcq_scores = pd.read_csv("../data/mcq_scores.tsv", sep="\t")
mcq_scores["class"] = mcq_scores[["kg"]].apply(std_classification)

essay_scores = pd.read_csv("../data/essay_scores.csv")
essay_scores["class"] = essay_scores[["kg"]].apply(std_classification)

In [37]:
mcq_scores[["class", "uid"]].groupby("class").count()

,uid
class,
0,69
1,46


In [38]:
essay_scores[["class", "uid"]].groupby("class").count()

,uid
class,
0,48
1,67


In [39]:
import re
pd.merge(mcq_scores[["uid", "pre", "post", "kg"]], essay_scores[["uid", "pre", "post", "kg"]], on="uid", how="inner").rename(columns=lambda x: re.sub("_x", "_mcq", x)).rename(columns=lambda x: re.sub("_y", "_essay", x)).drop(columns=["uid"]).corr().round(2)

,pre_mcq,post_mcq,kg_mcq,pre_essay,post_essay,kg_essay
pre_mcq,1.00,0.41,-0.61,0.41,0.13,-0.13
post_mcq,0.41,1.00,0.47,0.25,0.38,0.20
kg_mcq,-0.61,0.47,1.00,-0.17,0.21,0.30
pre_essay,0.41,0.25,-0.17,1.00,0.23,-0.40
post_essay,0.13,0.38,0.21,0.23,1.00,0.80
kg_essay,-0.13,0.20,0.30,-0.40,0.80,1.00


# Cole 2011 Features (MCQ Scores)

In [40]:
features_lightning = pd.read_csv("../out/cole2011_tracking_features.csv")
features = pd.merge(features_lightning, mcq_scores, on="uid", how="inner")

res = []

low = features[features["class"] == 0]
high = features[features["class"] == 1]
cols = features.drop(columns=["source", "task", "acode", "uid", "pre", "post", "kg", "class"], errors="ignore").columns

for col in cols:
    res.append([col, mannwhitneyu(low[col].dropna(), high[col].dropna())[1], low[col].median(), high[col].median(), f"{low[col].mean().round(2)} +- {low[col].std().round(2)}", f"{high[col].mean().round(2)} +- {high[col].std().round(2)}"])
    
df = pd.DataFrame(res, columns=["Feature Name", "M-W U (p-value)", "median(low)", "median(high)", "mean+-std(low)", "mean+-std(high)"])
df = df.round(3)
cole_mcq_results = df[["Feature Name", "M-W U (p-value)"]].rename(columns={"M-W U (p-value)": "cole_mcq_M-W U (p-value)"})
df

,Feature Name,M-W U (p-value),median(low),median(high),mean+-std(low),mean+-std(high)
0,session_sum_fixation_duration,0.247,1189322.650,997856.650,1162923.41 +- 578938.59,1024166.08 +- 496224.45
1,session_mean_fixation_duration,0.015,469.498,420.550,472.6 +- 87.25,435.89 +- 76.91
2,session_count_fixations,0.672,2420.500,2050.500,2499.13 +- 1306.41,2374.32 +- 1176.4
3,content-page_max_sum_reading_duration,0.020,201032.600,156985.200,238873.57 +- 154356.6,181305.06 +- 131548.96
4,content-page_mean_sum_reading_duration,0.026,77845.921,60270.280,107690.92 +- 104419.59,68959.19 +- 44462.0
5,session_mean_reading_sequence_duration,0.006,3249.654,2896.829,3428.52 +- 1019.71,2940.8 +- 808.87
6,session_count_reading_sequence,0.342,145.000,131.000,151.79 +- 86.59,135.18 +- 78.51
7,session_sum_reading_fixation_duration,0.063,468157.200,376139.900,507517.1 +- 282846.85,406637.99 +- 252800.63
8,session_mean_reading_fixation_duration,0.014,484.163,439.985,497.77 +- 115.51,446.67 +- 82.88
9,session_count_reading_fixation,0.290,933.500,865.500,1035.01 +- 599.78,909.75 +- 560.3


# Cole 2011 Features (Essay Scores)

In [41]:
features_lightning = pd.read_csv("../out/cole2011_tracking_features.csv")
features = pd.merge(features_lightning, essay_scores, on="uid", how="inner")

res = []

low = features[features["class"] == 0]
high = features[features["class"] == 1]
cols = features.drop(columns=["source", "task", "acode", "uid", "pre", "post", "kg", "class"], errors="ignore").columns

for col in cols:
    res.append([col, mannwhitneyu(low[col].dropna(), high[col].dropna())[1], low[col].median(), high[col].median(), f"{low[col].mean().round(2)} +- {low[col].std().round(2)}", f"{high[col].mean().round(2)} +- {high[col].std().round(2)}"])
    
df = pd.DataFrame(res, columns=["Feature Name", "M-W U (p-value)", "median(low)", "median(high)", "mean+-std(low)", "mean+-std(high)"])
df = df.round(3)
cole_essay_results = df[["Feature Name", "M-W U (p-value)"]].rename(columns={"M-W U (p-value)": "cole_essay_M-W U (p-value)"})
df

,Feature Name,M-W U (p-value),median(low),median(high),mean+-std(low),mean+-std(high)
0,session_sum_fixation_duration,0.045,847239.300,1181171.900,1008814.5 +- 624612.13,1183109.42 +- 477911.34
1,session_mean_fixation_duration,0.096,466.067,433.529,476.85 +- 99.12,444.18 +- 70.08
2,session_count_fixations,0.005,1880.500,2626.000,2101.08 +- 1282.78,2711.86 +- 1173.12
3,content-page_max_sum_reading_duration,0.661,173907.600,188174.200,215044.74 +- 159975.75,217166.84 +- 139502.27
4,content-page_mean_sum_reading_duration,0.895,70319.400,71421.568,91922.52 +- 72566.63,92889.15 +- 98245.59
5,session_mean_reading_sequence_duration,0.021,3238.608,2913.891,3513.57 +- 1100.84,3029.43 +- 804.45
6,session_count_reading_sequence,0.030,121.000,149.000,124.08 +- 75.19,161.16 +- 86.49
7,session_sum_reading_fixation_duration,0.310,384866.650,455790.600,439868.6 +- 286579.82,488899.09 +- 265895.15
8,session_mean_reading_fixation_duration,0.060,481.670,447.265,503.89 +- 127.04,458.05 +- 83.85
9,session_count_reading_fixation,0.079,819.500,948.500,859.75 +- 517.53,1080.34 +- 618.47


# Our Features (MCQ Scores)

In [42]:
features_lightning = pd.read_csv("../out/our_tracking_features.csv")
features = pd.merge(features_lightning, mcq_scores, on="uid", how="inner")

res = []

low = features[features["class"] == 0]
high = features[features["class"] == 1]
cols = features.drop(columns=["source", "task", "acode", "uid", "pre", "post", "kg", "class"], errors="ignore").columns

for col in cols:
    res.append([col, mannwhitneyu(low[col].dropna(), high[col].dropna())[1], low[col].median(), high[col].median(), f"{low[col].mean().round(2)} +- {low[col].std().round(2)}", f"{high[col].mean().round(2)} +- {high[col].std().round(2)}"])
    
df = pd.DataFrame(res, columns=["Feature Name", "M-W U (p-value)", "median(low)", "median(high)", "mean+-std(low)", "mean+-std(high)"])
df = df.round(3)
word_level_mcq_results = df[["Feature Name", "median(low)", "median(high)", "mean+-std(low)", "mean+-std(high)", "M-W U (p-value)"]].rename(columns={"M-W U (p-value)": "our_mcq_M-W U (p-value)"})
df

,Feature Name,M-W U (p-value),median(low),median(high),mean+-std(low),mean+-std(high)
0,session_sum_fixation_duration,0.255,918138.950,725317.150,944198.33 +- 513854.42,813216.97 +- 421938.14
1,session_mean_fixation_duration,0.018,468.788,425.188,472.41 +- 98.08,432.6 +- 70.67
2,session_count_fixations,0.625,1936.000,1736.500,2013.3 +- 1098.69,1888.68 +- 971.92
3,content-page_max_sum_reading_duration,0.075,253041.200,196265.400,311844.45 +- 210242.88,250945.84 +- 178624.73
4,content-page_mean_sum_reading_duration,0.133,105500.263,85322.568,137866.44 +- 132227.32,100807.16 +- 64331.3
5,session_mean_reading_sequence_duration,0.011,4149.901,3513.137,4398.68 +- 1704.48,3663.46 +- 1418.43
6,session_count_reading_sequence,0.899,141.500,143.000,157.08 +- 97.57,154.18 +- 84.8
7,session_sum_reading_fixation_duration,0.229,586035.550,517547.850,635323.32 +- 353797.99,546318.36 +- 305758.94
8,session_mean_reading_fixation_duration,0.015,475.150,431.587,490.38 +- 113.01,441.98 +- 80.97
9,session_count_reading_fixation,0.662,1229.500,1196.500,1309.27 +- 739.86,1236.65 +- 685.14


# Our Features (Essay Scores)

In [43]:
features_lightning = pd.read_csv("../out/our_tracking_features.csv")
features = pd.merge(features_lightning, essay_scores, on="uid", how="inner")

res = []

low = features[features["class"] == 0]
high = features[features["class"] == 1]
cols = features.drop(columns=["source", "task", "acode", "uid", "pre", "post", "kg", "class"], errors="ignore").columns

for col in cols:
    res.append([col, mannwhitneyu(low[col].dropna(), high[col].dropna())[1], low[col].median(), high[col].median(), f"{low[col].mean().round(2)} +- {low[col].std().round(2)}", f"{high[col].mean().round(2)} +- {high[col].std().round(2)}"])
    
df = pd.DataFrame(res, columns=["Feature Name", "M-W U (p-value)", "median(low)", "median(high)", "mean+-std(low)", "mean+-std(high)"])
df = df.round(3)
word_level_essay_results = df[["Feature Name", "M-W U (p-value)"]].rename(columns={"M-W U (p-value)": "our_essay_M-W U (p-value)"})
df

,Feature Name,M-W U (p-value),median(low),median(high),mean+-std(low),mean+-std(high)
0,session_sum_fixation_duration,0.118,660543.950,933663.450,837042.84 +- 554586.1,939029.97 +- 420295.52
1,session_mean_fixation_duration,0.059,475.882,430.670,480.04 +- 108.65,440.02 +- 69.7
2,session_count_fixations,0.021,1617.000,1986.000,1711.48 +- 1054.99,2161.62 +- 1011.2
3,content-page_max_sum_reading_duration,0.653,230853.300,245413.800,288316.64 +- 217809.33,289283.36 +- 187535.32
4,content-page_mean_sum_reading_duration,0.724,90926.856,97061.217,115849.35 +- 84472.47,130040.02 +- 130561.12
5,session_mean_reading_sequence_duration,0.042,4002.653,3698.871,4477.62 +- 1679.96,3848.01 +- 1559.26
6,session_count_reading_sequence,0.013,111.500,155.000,132.74 +- 91.29,173.8 +- 90.24
7,session_sum_reading_fixation_duration,0.201,480251.950,610269.150,564930.62 +- 364204.39,629954.42 +- 316240.92
8,session_mean_reading_fixation_duration,0.030,488.455,434.788,500.57 +- 124.6,450.3 +- 80.23
9,session_count_reading_fixation,0.045,1126.500,1362.000,1108.85 +- 644.62,1414.52 +- 746.6


# Compose results

In [44]:
results = pd.merge(word_level_mcq_results, word_level_essay_results, on="Feature Name", how="left")
results = pd.merge(results, cole_mcq_results, on="Feature Name", how="left")
results = pd.merge(results, cole_essay_results, on="Feature Name", how="left")
results

,Feature Name,median(low),median(high),mean+-std(low),mean+-std(high),our_mcq_M-W U (p-value),our_essay_M-W U (p-value),cole_mcq_M-W U (p-value),cole_essay_M-W U (p-value)
0,session_sum_fixation_duration,918138.950,725317.150,944198.33 +- 513854.42,813216.97 +- 421938.14,0.255,0.118,0.247,0.045
1,session_mean_fixation_duration,468.788,425.188,472.41 +- 98.08,432.6 +- 70.67,0.018,0.059,0.015,0.096
2,session_count_fixations,1936.000,1736.500,2013.3 +- 1098.69,1888.68 +- 971.92,0.625,0.021,0.672,0.005
3,content-page_max_sum_reading_duration,253041.200,196265.400,311844.45 +- 210242.88,250945.84 +- 178624.73,0.075,0.653,0.020,0.661
4,content-page_mean_sum_reading_duration,105500.263,85322.568,137866.44 +- 132227.32,100807.16 +- 64331.3,0.133,0.724,0.026,0.895
5,session_mean_reading_sequence_duration,4149.901,3513.137,4398.68 +- 1704.48,3663.46 +- 1418.43,0.011,0.042,0.006,0.021
6,session_count_reading_sequence,141.500,143.000,157.08 +- 97.57,154.18 +- 84.8,0.899,0.013,0.342,0.030
7,session_sum_reading_fixation_duration,586035.550,517547.850,635323.32 +- 353797.99,546318.36 +- 305758.94,0.229,0.201,0.063,0.310
8,session_mean_reading_fixation_duration,475.150,431.587,490.38 +- 113.01,441.98 +- 80.97,0.015,0.030,0.014,0.060
9,session_count_reading_fixation,1229.500,1196.500,1309.27 +- 739.86,1236.65 +- 685.14,0.662,0.045,0.290,0.079
